In [ ]:
def sweep():
    with wandb.init(project="sirl_optimizer"):
        config = wandb.config

        # Walk-forward validation 내부 루프
        all_pv = [] # 모든 phase의 평균 pv 모음
        for phase, train_idx, test_idx in enumerate(walk_forward_split(data)):
            all_pv_per_phase = [] # 한 phase의 n회치 pv 모음
            for _ in range(5):
                learner.run()
                pv = test_learner.run()
                all_pv_per_phase.append(pv)
            avg_pv_per_phase = sum(all_pv_per_phase) / len(all_pv_per_phase)
            
            all_pv.append(avg_pv_per_phase)
        avg_pv = sum(all_pv) / len(all_pv)

        # 하이퍼파라미터 최적화 과정에서 pv값의 변화 기록
        for phase, pv in enumerate(all_pv):
            wandb.log({f"phase{phase}": pv})
        wandb.log({"walk_forward_score": avg_pv})

In [2]:
# 2: 검색 공간 정의하기
sweep_config = {
    "method": "bayes",
    "metric": {
        "goal": "maximize",
        "name": "walk_forward_score"
    },
    "parameters": {
        "lr": {"min": 0.001, "max": 0.1},
        "window_size": {"values": [10, 15, 20]},
        "feature_window": {"values": [10, 15, 20]},
        "num_step": {"values": [1, 2, 4, 8]},
        "noise": {"min": 0.1, "max": 0.9},
        "max_episode": {"min": 10, "max": 30, "distribution": "int_uniform"}
    },
}

# 3: 스윕 시작하기
sweep_id = wandb.sweep(sweep=sweep_configuration, project="my-first-sweep")

wandb.agent(sweep_id, function=sweep, count = 10)

SyntaxError: unterminated string literal (detected at line 9) (2531570642.py, line 9)